#Data Loading

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask
import dask.dataframe as dd

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
y_train = pd.read_excel('/content/gdrive/MyDrive/Y_train.xlsx')
x_train = dd.read_csv('/content/gdrive/MyDrive/DS_train(2020-06--2022-06-01).csv', sep='\\t', blocksize=64000000)
x_train = x_train.iloc[:,[0,1,2,3]]

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
x_train_with_only_instock = x_train.dropna()
x_train2 = x_train_with_only_instock[x_train_with_only_instock['DateObserve'] > '2022-03-28']
pd_train2 = x_train2.compute()

In [ ]:
obsevations1 = pd_train2.groupby(by='WebPriceId').DateObserve.nunique()

In [ ]:
obsevations = obsevations1[(obsevations1 >= 4)]

In [ ]:
goods_means = pd_train2[pd_train2['WebPriceId'].isin(obsevations.index)].groupby(by='WebPriceId').CurrentPrice.mean() #mean price for each one Id

In [ ]:
technique = goods_means[goods_means > goods_means.quantile(0.75)].index #0.75 quantile

In [ ]:
people_goods = goods_means[goods_means <= goods_means.quantile(0.25)].index #0.25 quantile

In [ ]:
other_goods = goods_means[(goods_means > goods_means.quantile(0.25)) & (goods_means <= goods_means.quantile(0.75))].index #from 0.25 to 0.75 quantile

In [ ]:
indexes = np.concatenate([np.concatenate([np.array(people_goods), np.array(technique)]), np.array(other_goods)])

In [ ]:
dataset = x_train_with_only_instock[x_train_with_only_instock['WebPriceId'].isin(indexes)]

dataset = dataset.drop('StockStatus', axis=1) 

dataset = dataset.compute()

dataset['DateObserve'] = pd.to_datetime(dataset['DateObserve'], yearfirst=True).dt.to_period('h')
  
dataset.to_csv('Train.csv', index=False)

#Data Preprocessing

In [ ]:
train_data = pd.read_csv('Train.csv')
train_data['DateObserve'] = pd.to_datetime(train_data['DateObserve'], yearfirst=True).dt.to_period('D')

people_ids = people_goods
technique_ids = technique
other_ids = other_goods

people = train_data[train_data['WebPriceId'].isin(people_ids)].drop('WebPriceId', axis=1)
technique = train_data[train_data['WebPriceId'].isin(technique_ids)].drop('WebPriceId', axis=1)
other = train_data[train_data['WebPriceId'].isin(other_ids)].drop('WebPriceId', axis=1)

people = people.groupby(by='DateObserve').CurrentPrice.mean().to_frame().reset_index()
technique = technique.groupby(by='DateObserve').CurrentPrice.mean().to_frame().reset_index()
other = other.groupby(by='DateObserve').CurrentPrice.mean().to_frame().reset_index()

people = people.rename(columns={'CurrentPrice': 'PeoplePrice'})
technique = technique.rename(columns={'CurrentPrice': 'TechniquePrice'})
other = other.rename(columns={'CurrentPrice': 'OtherPrice'})

In [ ]:
global_set = train_data.groupby(by='DateObserve').CurrentPrice.mean().to_frame().reset_index()
global_set = pd.merge(global_set, people, on=['DateObserve'], how='left')
global_set = pd.merge(global_set, technique, on=['DateObserve'], how='left')
global_set = pd.merge(global_set, other, on=['DateObserve'], how='left')

global_set = global_set.fillna({'PeoplePrice': global_set.PeoplePrice.median(), 
                   'TechniquePrice': global_set.TechniquePrice.median(),
                   'OtherPrice': global_set.OtherPrice.median(),})

In [ ]:
global_set = global_set.assign(Date = lambda x: pd.to_datetime(x['DateObserve'].dt.strftime('%Y-%m')))

global_set.loc[global_set['Date'].isin(['2020-06-01']), 'Y'] = 0.28
global_set.loc[global_set['Date'].isin(['2020-07-01']), 'Y'] = -0.065
global_set.loc[global_set['Date'].isin(['2020-08-01']), 'Y'] = -0.005
global_set.loc[global_set['Date'].isin(['2020-09-01']), 'Y'] = 0.315
global_set.loc[global_set['Date'].isin(['2020-10-01']), 'Y'] = 0
global_set.loc[global_set['Date'].isin(['2020-11-01']), 'Y'] = 0.375
global_set.loc[global_set['Date'].isin(['2020-12-01']), 'Y'] = 0.35
global_set.loc[global_set['Date'].isin(['2021-01-01']), 'Y'] = 0.38
global_set.loc[global_set['Date'].isin(['2021-02-01']), 'Y'] = 0.685
global_set.loc[global_set['Date'].isin(['2021-03-01']), 'Y'] = 0.255
global_set.loc[global_set['Date'].isin(['2021-04-01']), 'Y'] = 0.13
global_set.loc[global_set['Date'].isin(['2021-05-01']), 'Y'] = 0.485
global_set.loc[global_set['Date'].isin(['2021-06-01']), 'Y'] = 0.25
global_set.loc[global_set['Date'].isin(['2021-07-01']), 'Y'] = -0.135
global_set.loc[global_set['Date'].isin(['2021-08-01']), 'Y'] = 0.35
global_set.loc[global_set['Date'].isin(['2021-09-01']), 'Y'] = 0.345
global_set.loc[global_set['Date'].isin(['2021-10-01']), 'Y'] = 0.675
global_set.loc[global_set['Date'].isin(['2021-11-01']), 'Y'] = 0.74
global_set.loc[global_set['Date'].isin(['2021-12-01']), 'Y'] = 1.255
global_set.loc[global_set['Date'].isin(['2022-01-01']), 'Y'] = 0.355
global_set.loc[global_set['Date'].isin(['2022-02-01']), 'Y'] = 1.355
global_set.loc[global_set['Date'].isin(['2022-03-01']), 'Y'] = 3.555
global_set.loc[global_set['Date'].isin(['2022-04-01']), 'Y'] = 1.89
global_set.loc[global_set['Date'].isin(['2022-05-01']), 'Y'] = 1.56

global_set = global_set.drop('Date', axis=1)

In [ ]:
global_set = global_set.set_index('DateObserve')

global_set['Ind'] = np.arange(global_set.shape[0])
for i in range(1, 5):
  global_set[f'ShiftCP{i}'] = global_set['CurrentPrice'].shift(i)
  global_set[f'ShiftPP{i}'] = global_set['PeoplePrice'].shift(i)
  global_set[f'ShiftTP{i}'] = global_set['TechniquePrice'].shift(i)
  global_set[f'ShiftOP{i}'] = global_set['OtherPrice'].shift(i)
global_set = global_set['2020-06-26':]

for i in global_set.columns:
  if i in ['CurrentPrice', 'PepoplePrice', 'OtherPrice', 'TechniquePrice']: 
    global_set[f'MA{i}'] = global_set[i].rolling(
      window=30,       
      center=True,      
      min_periods=15).median()

#Model

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch 
import torch.nn as nn
from torch.autograd import Variable 

X = global_set.drop('Y', axis=1)
y = global_set.Y 

mm = MinMaxScaler()
ss = StandardScaler()

for_train = 300

X_ss = ss.fit_transform(X)
y_mm = mm.fit_transform(np.array(y).reshape(-1, 1)) 

X_train = X_ss[:for_train, :]
X_test = X_ss[for_train:, :]

y_train = y_mm[:for_train, :]
y_test = y_mm[for_train:, :] 


X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test)) 

X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))
X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1])) 

In [ ]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes 
        self.num_layers = num_layers 
        self.input_size = input_size 
        self.hidden_size = hidden_size 
        self.seq_length = seq_length 

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True)
        self.fc_1 =  nn.Linear(hidden_size, 128) 
        self.fc = nn.Linear(128, num_classes) 

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        hn = hn.view(-1, self.hidden_size)
        out = self.relu(hn)
        out = self.fc_1(out)
        out = self.relu(out)
        out = self.fc(out)
        return out

In [ ]:
num_epochs = 1000
learning_rate = 0.001

input_size = 24 
hidden_size = 10 
num_layers = 1 

num_classes = 1 

lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1])

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate) 

In [ ]:
for epoch in range(num_epochs):
  outputs = lstm1.forward(X_train_tensors_final)
  optimizer.zero_grad()
  loss = criterion(outputs, y_train_tensors)
  loss.backward()
  optimizer.step()

Epoch: 0, loss: 0.16354
Epoch: 100, loss: 0.00191
Epoch: 200, loss: 0.00097
Epoch: 300, loss: 0.00060
Epoch: 400, loss: 0.00043
Epoch: 500, loss: 0.00035
Epoch: 600, loss: 0.00028
Epoch: 700, loss: 0.00022
Epoch: 800, loss: 0.00018
Epoch: 900, loss: 0.00014


In [ ]:
df_X_ss = ss.transform(global_set.drop('Y', axis=1)) 
df_y_mm = mm.transform(np.array(global_set.Y).reshape(-1, 1))

df_X_ss = Variable(torch.Tensor(df_X_ss)) 
df_y_mm = Variable(torch.Tensor(df_y_mm))

df_X_ss = torch.reshape(df_X_ss, (df_X_ss.shape[0], 1, df_X_ss.shape[1])) 

train_predict = lstm1(df_X_ss)
df_X_ss = ss.transform(global_set.drop('Y', axis=1)) 
df_y_mm = mm.transform(np.array(global_set.Y).reshape(-1, 1)) 

data_predict = train_predict.data.numpy()
data_predict = train_predict.data.numpy() 
data_predict = mm.inverse_transform(data_predict) 

from sklearn.metrics import mean_absolute_error
mean_absolute_error(data_predict, global_set.Y)